In [1]:
from essential_takehome.files import load_dataframes
datasets = load_dataframes()

2023-11-29 14:10:04.885 | INFO     | essential_takehome.files:load_dataframes:22 - Loaded file country_id_mapping.csv
2023-11-29 14:10:04.962 | INFO     | essential_takehome.files:load_dataframes:22 - Loaded file customer_data.csv
2023-11-29 14:10:05.496 | INFO     | essential_takehome.files:load_dataframes:22 - Loaded file session_clickstream_data.csv
2023-11-29 14:10:05.518 | INFO     | essential_takehome.files:load_dataframes:22 - Loaded file sku_data.csv


# Test Agent Output

In [2]:
import pandas as pd

# Step 1: Merge the necessary dataframes
customer_data = datasets['customer_data.csv']
session_clickstream_data = datasets['session_clickstream_data.csv']
sku_data = datasets['sku_data.csv']

merged_data = pd.merge(session_clickstream_data, customer_data, on='Customer ID')
merged_data = pd.merge(merged_data, sku_data, on='SKU ID')

# Step 2: Analyze product features and session activity
product_features = ['Available Colors', 'Materials', 'Type']
session_activity = ['time spent', 'avg purchases']

product_features_data = merged_data[product_features]
session_activity_data = merged_data[session_activity]

# Step 3: Calculate trends in product features and session activity
product_features_trends = product_features_data.mode().iloc[0]
session_activity_trends = session_activity_data.mean()

# Step 4: Display the trends
print("Product Features Trends:")
print(product_features_trends)
print("\nSession Activity Trends:")
print(session_activity_trends)

KeyError: "None of [Index(['time spent', 'avg purchases'], dtype='object')] are in the [columns]"

# Manually Defined Answer

In [17]:
import pandas as pd

# Step 1: Merge the necessary dataframes
# customer_data = datasets['customer_data.csv']
session_clickstream_data = datasets['session_clickstream_data.csv']
sku_data = datasets['sku_data.csv']

# Assuming the session data has year, month, and day columns
session_clickstream_data['Date'] = pd.to_datetime(session_clickstream_data[['year', 'month', 'day']])

# Group by 'Date' and 'SKU ID' (adjust the column names based on your dataset)
grouped_session_data = session_clickstream_data.groupby(['Date', 'SKU ID']).agg(
    total_rows=pd.NamedAgg(column='SKU ID', aggfunc='size'),  # Count the number of rows in each group
    purchased_count=pd.NamedAgg(column='Purchased', aggfunc=lambda x: (x == True).sum()),  # Count where 'Purchased' is True
    total_customers_count=pd.NamedAgg(column='Customer ID', aggfunc='count')
)


merged_data = pd.merge(grouped_session_data.reset_index(), sku_data, on='SKU ID')
# print(merged_data.head())

grouped_merged_data = merged_data.groupby(['Available Colors', 'Type', 'Materials', 'Date']).agg(
    total_rows_sum=pd.NamedAgg(column='total_rows', aggfunc='sum'),
    purchased_count_sum=pd.NamedAgg(column='purchased_count', aggfunc='sum')
    # Add other aggregations as needed
)
# grouped_merged_data

pivot_data = merged_data.pivot_table(
    index='Date',
    columns=['Available Colors', 'Type', 'Materials'],
    values='purchased_count',
    aggfunc='sum'
)
# Calculate the day-to-day difference in 'purchased_count_sum'
daily_difference = pivot_data.diff()

# Check if the values are increasing or decreasing
trend_analysis = daily_difference.applymap(lambda x: 'Increasing' if x > 0 else ('Decreasing' if x < 0 else 'No Change'))
print(trend_analysis)



Available Colors       Black                                                 \
Type                  Blouse                              Dress               
Materials             Cotton   Polyester       Wool      Cotton       Denim   
Date                                                                          
2008-04-01         No Change   No Change  No Change   No Change   No Change   
2008-04-02        Decreasing  Increasing  No Change   No Change  Increasing   
2008-04-03        Decreasing   No Change  No Change  Decreasing  Decreasing   
2008-04-04        Increasing   No Change  No Change  Decreasing  Decreasing   
2008-04-05        Increasing  Increasing  No Change  Increasing   No Change   
...                      ...         ...        ...         ...         ...   
2008-08-09         No Change   No Change  No Change   No Change  Increasing   
2008-08-10        Increasing   No Change  No Change   No Change  Decreasing   
2008-08-11        Increasing  Increasing  No Change 

/var/folders/82/nl12mr453xx9ltvm037413d80000gn/T/ipykernel_57509/3569038277.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  trend_analysis = daily_difference.applymap(lambda x: 'Increasing' if x > 0 else ('Decreasing' if x < 0 else 'No Change'))
